<h1 style="font-family: 'Comic Sans MS'">
    📄 Azure AI Document Intelligence 
</h1>


<h2 style="font-family: 'Comic Sans MS'">
    🚀 Deploying using a bicep file 
</h2>

1. To log into you Azure account: `az login` 
2. Choose which subscription you want to use: `az account set --subscription <your-subscription-id>`
3. Create a resource group that will contain the resources needed to run this notebook: `az group create --name {resource_group_name} --location {location}`
    - For a list of regions: https://gist.github.com/ausfestivus/04e55c7d80229069bf3bc75870630ec8



= 'S0'
```

```python
# Import the os module to run shell commands
import os

# Log in to Azure
os.system('az login')

# Define the resource group name and location
resource_group_name = 'myResourceGroup'
location = 'eastus'

# Create the resource group
os.system(f'az group create --name {resource_group_name} --location {location}')

------



<h2 style="font-family: 'Comic Sans MS'">
    📦 Prebuilt models 
</h2>

<h3 style="font-family: 'Comic Sans MS'">
    Analysing contracts using the Contract model
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Analysing general documents using the General document model
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Analysing invoices using the Invoice model
</h3>

In [ ]:
import requests
import json

# Replace with your Azure Document Intelligence endpoint and API key
endpoint = "https://eastus.api.cognitive.microsoft.com/"
api_key = "your_api_key_here"

# URL of the prebuilt model endpoint
prebuilt_model_url = f"{endpoint}/formrecognizer/documentModels/prebuilt-invoice:analyze?api-version=2023-02-28-preview"

# Path to the document you want to analyze
document_path = "pdfcoffee.com_invoice-amazonpdf-pdf-free.pdf"

# Read the document content
with open(document_path, "rb") as file:
    document_content = file.read()

# Set the headers
headers = {
    "Content-Type": "application/pdf",
    "Ocp-Apim-Subscription-Key": api_key
}

# Make the request to analyze the document
response = requests.post(prebuilt_model_url, headers=headers, data=document_content)

# Check the response status
if response.status_code == 202:
    # Get the operation-location URL from the response headers
    operation_location = response.headers["Operation-Location"]

    # Poll the operation-location URL to get the result
    result_response = requests.get(operation_location, headers={"Ocp-Apim-Subscription-Key": api_key})

    # Check if the result is ready
    while result_response.status_code == 200 and result_response.json().get("status") != "succeeded":
        result_response = requests.get(operation_location, headers={"Ocp-Apim-Subscription-Key": api_key})

    # Print the result
    if result_response.status_code == 200:
        result = result_response.json()
        print(json.dumps(result, indent=4))
    else:
        print(f"Failed to get the result: {result_response.status_code}")
else:
    print(f"Failed to analyze the document: {response.status_code}")

{
    "status": "succeeded",
    "createdDateTime": "2025-02-23T12:46:19Z",
    "lastUpdatedDateTime": "2025-02-23T12:46:26Z",
    "analyzeResult": {
        "apiVersion": "2023-02-28-preview",
        "modelId": "prebuilt-invoice",
        "stringIndexType": "textElements",
        "content": "amazon.in\nTax Invoice/Bill of Supply/Cash Memo\n(Original for Recipient)\nSold By :\nBilling Address :\nDarshita Aashiyana Private Limited\nkarthik\n*\nUnit No. 1, Khewat/ Khata No: 373/ 400 Mustatil\nC1001 ace city, sector1\nNo 31,, Village Taoru, Tehsil Taoru, District\nGREATER NOIDA, UTTAR PRADESH, 201306\nMewat,, On Bilaspur Taoru Road\nIN\nMewat, Haryana, 122105\nIN\nShipping Address :\nkarthik\nPAN No: AAFCD6883Q\nkarthik\nGST Registration No: 06AAFCD6883Q1ZU\nC1001 ace city, sector1\nGREATER NOIDA, UTTAR PRADESH, 201306\nIN\nOrder Number: 407-8153595-7245952\nInvoice Number : DEL2-68786\nOrder Date: 10.08.2019\nInvoice Details : HR-DEL2-179184911-1920\nInvoice Date : 10.08.2019\nSl.\nDes

In [30]:
if result_response.status_code == 200:
    result = result_response.json()
    result_dict = json.loads(json.dumps(result))  # Convert JSON to dictionary
    print(result_dict["analyzeResult"].keys())
    # print(result_dict["analyzeResult"]["keyValuePairs"][6])
else:
    print(f"Failed to get the result: {result_response.status_code}")

dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'tables', 'keyValuePairs', 'styles', 'documents'])


In [37]:
if result_response.status_code == 200:
    result = result_response.json()
    result_dict = json.loads(json.dumps(result))  # Convert JSON to dictionary
    print(result_dict["analyzeResult"]["tables"][0]["cells"][0])
    
else:
    print(f"Failed to get the result: {result_response.status_code}")

{'kind': 'columnHeader', 'rowIndex': 0, 'columnIndex': 0, 'content': 'Sl.\nNo', 'boundingRegions': [{'pageNumber': 1, 'polygon': [0.5878, 4.6109, 0.8432, 4.6109, 0.8432, 4.9393, 0.5878, 4.9393]}], 'spans': [{'offset': 667, 'length': 3}, {'offset': 720, 'length': 2}]}


<h3 style="font-family: 'Comic Sans MS'">
    Analysing the document structure layout using the Layout model
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Doing simple OCR using the Read model
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Analysing receipts using the Receipt model
</h3>

<h2 style="font-family: 'Comic Sans MS'">
    🏗️ Custom models 
</h2>

You typically want to use custom models when prebuilt models don't yield good enough results. 
</br>
- There are 2 types of custom document models:
    - Custom template (requires at least 5 examples to start training from scratch)
    - Custom neural (pretrained model that you can finetune on your own data)
- It's advised to start with a custom neural model.


<h3 style="font-family: 'Comic Sans MS'">
    Composed models
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Incremental classifiers (incremental training)
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Custom template models
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Custom neural models
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Custom classification models
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Custom labels
</h3>

<h2 style="font-family: 'Comic Sans MS'">
    🧩 Add-on features
</h2>

<h3 style="font-family: 'Comic Sans MS'">
    High resolution extraction
</h3>

In [ ]:
import requests

# Replace with your actual endpoint and subscription key
endpoint = "https://eastus.api.cognitive.microsoft.com/"
subscription_key = "9f57a65e1d724c16bed7f4972ecc8370"
url = f"{endpoint}.cognitiveservices.azure.com/documentintelligence/documentModels/prebuilt-layout:analyze?api-version=2024-02-29-preview&features=ocrHighResolution"
# Replace with your document URL
document_url = "https://asdfsdafasd.blob.core.windows.net/dfsgdfgsfd/pdfcoffee.com_invoice-amazonpdf-pdf-free.pdf"

headers = {
    "Ocp-Apim-Subscription-Key": subscription_key,
    "Content-Type": "application/json"
}

data = {
    "urlSource": document_url
}

response = requests.post(url, headers=headers, json=data)

if response.status_code == 200:
    print("Response:")
    print(response.json())
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

Request failed with status code: 404
{"error":{"code":"404","message": "Resource not found"}}


<h3 style="font-family: 'Comic Sans MS'">
    Barcode property extraction
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Language detection
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Searchable PDF
</h3>

<h3 style="font-family: 'Comic Sans MS'">
    Key-value Pairs
</h3>


<h3 style="font-family: 'Comic Sans MS'">
    Query Fields
</h3>